In [ ]:
import os 
import glob 
from pathlib import Path
import numpy as np 
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt 
import seaborn as sns 

In [ ]:
# Cargar una muestra de access log file 
n_samples = 10
log_file_path = '../data/target/access_log_test.log'
with open(log_file_path, 'r') as f:
  sample_lines = [next(f).strip() for _ in range(n_samples)]
print("Sample log lines:")
for i,line in enumerate(sample_lines, 1):
  print(f"{i:>{len(str(n_samples))}}) {line}") 

### Parsing Access Log File

Existen varias formas de parsear access logs en un formato estructurado. Entre las formas se encuentra: `lars` (Herramienta especializada para logs de servidores web como Apache, Nginx, IIS. Esta herramienta no requiere escribir patrones), `parse` (Alternativa a Regex y esta en un punto intermedio entre la herramienta `lars` y `re`, ya que debe diseñarse patrones pero estos son más legibles que Regex) y `re`/**Regex** (enfoque basado en patrones de expresiones regulares personalizados, pero este requiere escribir y mantener patrones complejos para el correcto parseo)  

#### Uso de la biblioteca `lars`

In [ ]:
from lars.apache import ApacheSource

with open(log_file_path) as f:
  with ApacheSource(f) as source:
    for i,row in enumerate(source, 1):
      print(f"{i:>{len(str(n_samples))}}) {row}") 

In [ ]:
from lars.apache import ApacheSource, COMBINED

parsed_logs = []
with open(log_file_path) as f:
  with ApacheSource(f, log_format=COMBINED) as source:
    for i,row in enumerate(source, 1):
      print(f"{i:>{len(str(n_samples))}}) {row}") 
      
      record = {
        "remote_host" : row.remote_host,
        "ident" : row.ident,
        "remote_user" : row.remote_user, 
        "time" : row.time,
        "request_method" : row.request.method,
        "request_url_scheme" : row.request.url.scheme, 
        "request_url_netloc" : row.request.url.netloc,
        "request_url_path_str" : row.request.url.path_str,
        "request_url_params" : row.request.url.params, 
        "request_url_query_str" : row.request.url.query_str,
        "request_url_fragment" : row.request.url.fragment,
        "request_protocol": row.request.protocol,
        "status" : row.status,
        "size" : row.size,
        "req_Referer" : row.req_Referer,
        "req_User_agent" : row.req_User_agent
      }
      parsed_logs.append(record)

df_lars = pd.DataFrame(parsed_logs)
display(df_lars.head(5))

Los resultados muestran que al aplicar `ApacheSource` con los parámetros por defecto se tiene: 
- Parsea correctamente: IP del Cliente (`remote_host`), Fecha-Hora (`time`), Método (`request/method`), Recurso Solicitado (`request/url`), Protocolo (`request/protocol`), Código de Estado (`status`), Tamaño de Respuesta (`size`)
- Falla al parsear: User-Agent y Referrer. 

Pero, si se ajustan los parámetros y se modifica `log_format=COMBINED` (por defecto: `log_format=COMMON`) parsea correctamente todo el log. 

Los logs objetivos no siguen un formato W3C Extended ([W3C Extended Log File Format](https://www.w3.org/TR/WD-logfile.html)) por lo que aplicar `IISSource` no tendría sentido. 

---

```python
from lars.iis import IISSource

with open(log_file_path) as f:
  with IISSource(f) as source:
    for i,row in enumerate(source, 1):
      print(f"{i:>{len(str(n_samples))}}) {row}") 
```
> IISVersionError: Missing #Version directive before data

#### Uso de `parse`

In [ ]:
from parse import parse 

pattern = '{ip_client} {ident} {auth_user} [{timestamp}] "{method} {request} {protocol}" {status:d} {size:d} "{referrer}" "{agent}"'

parsed_logs = []
with open(log_file_path, 'r') as f:
  for line in f:
    parsed = parse(pattern, line.strip())
    parsed_logs.append(parsed.named)

for i,parsed in enumerate(parsed_logs, 1):
  print(f"{i:>{len(str(n_samples))}}) {parsed}") 

df_parse = pd.DataFrame(parsed_logs)
display(df_parse.head(5))

#### Uso de `re`/Regex

In [ ]:
import re 

log_pattern = r'(\S+) (\S+) (\S+) \[([^\]]+)\] "(\S+) ([^"]*) (\S+)" (\d+) (\d+|-) "([^"]*)" "([^"]*)"'

def parse_log_line(line):
  match = re.match(log_pattern, line)
  if not match:
    raise ValueError(f"Failed to parse log line: {line}")
  ip_client, ident, auth_user, timestamp, method, url, protocol, status, size, referer, user_agent = match.groups()
  return {
    'ip_client': ip_client,
    'ident': ident, 
    'auth_user': auth_user, 
    'timestamp': timestamp,
    'method': method,
    'url': url,
    'protocol': protocol,
    'status': int(status),
    'size': 0 if size == '-' else int(size),
    'referer': referer,
    'user_agent': user_agent
  }

# Parsea los logs del archivo utilizando el patrón anterior
parsed_logs = []
with open(log_file_path, 'r') as f:
  for line in f:
    parsed = parse_log_line(line.strip())
    if parsed:
      parsed_logs.append(parsed)

df_regex = pd.DataFrame(parsed_logs)
display(df_regex.head(5))

**Patrón de Expresión Regular para el Parsing de Logs**: `r'(\S+) (\S+) (\S+) \[([^\]]+)\] "(\S+) ([^"]*) (\S+)" (\d+) (\d+|-) "([^"]*)" "([^"]*)"'`
- `(\S+)`: IP del Cliente, Usuario (identidad del cliente, en la mayoría de los logs es `-`) y Campo Adicional (normalmente es el nombre del usuario autentificado, aunque, en la mayoría de los logs es `-`)
- `\[([^\]]+)\]`: Fecha-Hora (dentro de corchetes)
- `"(\S+) ([^"]*) (\S+)"`: Método-URL-Protocolo (dentro de comillas, separados por espacios)
  - `(\S+)`: Método (GET, POST, etc.)
  - `([^"]*)`: URL
  - `(\S+)`: Protocolo (HTTP/1.0, HTTP/1.1, etc.)
- `(\d+)`: Código de estado (`200`, `404`, `500`, etc.)
- `(\d+|-)`: Bytes Transferidos (número o `-` para cero)
- `"([^"]*)"`: Referencia (Referer) y Agente-Usuario (User-Agent)

Esta expresión regular parsea completamente el formato **Combined Log Format** de Apache (por lo que esta expresión parsea los mismos logs que parsea `ApacheSource` con `log_format=COMBINED`)

### Create Master Access Log

In [ ]:
input_pattern = "../data/target/access_log_*.log"
output_master = "access_log_master.log"
remove_duplicates = True 

files = sorted(
  glob.glob(input_pattern), 
  key=lambda x: [int(c) if c.isdigit() else c for c in re.split(r'(\d+)', x)]
)
lines_written = 0
unique_lines = set() if remove_duplicates else None 

with open(output_master, 'w') as outfile:
  for file in files:
    print(f" Processing: {file}")
    with open(file, 'r') as infile:
      for line in infile:
        if remove_duplicates:
          if line not in unique_lines:
            unique_lines.add(line)
            outfile.write(line)
            lines_written += 1
        else:
          outfile.write(line)
          lines_written += 1

print(f"Total de Líneas Escritas: {lines_written}")
if remove_duplicates:
  print(f"Líneas únicas conservadas: {len(unique_lines) if unique_lines else lines_written}")